In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

## Set up

In [20]:
import os
import tqdm
from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.multioutput import RegressorChain
from sklearn.base import BaseEstimator

import catboost as cb
import lightgbm as lgbm
import xgboost as xgb

import tensorflow as tf

from utils import load_config
from src.helpers import ContiguousGroupKFold

## Constants

In [4]:
config = load_config()

In [5]:
DATA_DIR = config['final_data']
TRAIN = os.path.join(DATA_DIR, 'train.csv')

## Modelling

- Direct Regressor: Predicts outputs and prediction interval directly
  - GBT with quantile loss
  - deep learning with quantile loss
- Ensemble Regressor: Uses a monte-carlo simulation to generate prediction interval
  - GBT with ensemble
  - deep learning with ensemble
- CV ensemble modelling - fold cross validation

In [ ]:
LOCATION = ['longitude', 'latitude']
CATEGORICAL = ['station_code', 'river', 'hydro_region', 'hydro_sector', 'hydro_sub_sector', 'hydro_zone']
NUM_STATION = ['altitude', 'catchment']
NUM_SOIL = ['bdod', 'cfvo', 'clay', 'sand']
NUM_METEO = ['tp', 't2m', 'swvl1', 'evap']
DISCHARGE = 'discharge'
DATE = 'ObsDate'

In [6]:
CV_SPLIT = 5

#### Load data

In [7]:
df = pd.read_csv(TRAIN)

/tmp/ipykernel_297326/2340250852.py:1: DtypeWarning: Columns (19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(TRAIN)


### Train Test Split
- Before generating rolling and lag features, we split the data to prevent data leakage.
- Based on previous analysis, the dataset exhibits annual seasonality but no significant long-term trend.
- Therefore, it is acceptable to use chronological or block-wise splits without always reserving the most recent data for validation.
- This approach is appropriate for seasonally-repeating time series, where the assumption of trend-driven data drift does not hold.
- In such cases, the model's ability to generalize across seasonal cycles is more important than strict recency.

In [8]:
cgkf = ContiguousGroupKFold(5)
for idx, (train_ids, val_ids) in enumerate(cgkf.split(df, groups = df.year)):
    print(f'Years in fold {idx + 1}')
    print('Train:', *df.iloc[train_ids].year.unique())
    print('Validation:', *df.iloc[val_ids].year.unique())
    print('-------------------------------------------------------------------------')

Years in fold 1
Train: 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
Validation: 1990 1991 1992
-------------------------------------------------------------------------
Years in fold 2
Train: 1990 1991 1992 1996 1997 1998 1999 2000 2001 2002 2003 2004
Validation: 1993 1994 1995
-------------------------------------------------------------------------
Years in fold 3
Train: 1990 1991 1992 1993 1994 1995 1999 2000 2001 2002 2003 2004
Validation: 1996 1997 1998
-------------------------------------------------------------------------
Years in fold 4
Train: 1990 1991 1992 1993 1994 1995 1996 1997 1998 2002 2003 2004
Validation: 1999 2000 2001
-------------------------------------------------------------------------
Years in fold 5
Train: 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001
Validation: 2002 2003 2004
-------------------------------------------------------------------------


### Models

In [10]:
np.sqrt(25 * 25 * 25)

125.0

In [18]:
import tensorflow as tf

def create_sequence_generator_model(cat_input_shapes, cont_input_shape, output_shape, embedding_dim=8, gru_n_layers = 2, gru_units=64, dropout = 0.2):
    """
    cat_input_shapes: list of vocab sizes for each categorical input
    cont_input_shape: number of continuous features (not a sequence)
    output_steps: number of sequential outputs to generate
    """
    # Inputs
    cont_input = tf.keras.layers.Input(shape=(cont_input_shape,), name='continuous_input')
    
    cat_inputs = []
    cat_embeddings = []

    for i, vocab_size in enumerate(cat_input_shapes):
        input_i = tf.keras.layers.Input(shape=(1,), name=f'cat_input_{i}')
        embedding_i = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, name=f'embedding_{i}')(input_i)
        embedding_i = tf.keras.layers.Reshape((embedding_dim,))(embedding_i)
        cat_inputs.append(input_i)
        cat_embeddings.append(embedding_i)

    # Concatenate all features (static input)
    x = tf.keras.layers.Concatenate()(cat_embeddings + [cont_input])  # shape: (batch, features)

    # Project to GRU dimension
    x = tf.keras.layers.Dense(gru_units, use_bias = False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(gru_units, activation='relu')(x)

    # Repeat vector to create initial sequence input for GRU
    x = tf.keras.layers.RepeatVector(output_shape)(x)  # shape: (batch, output_steps, gru_units)

    # GRU layers to model temporal dependency in output
    x = tf.keras.layers.GRU(gru_units, return_sequences=True)(x)
    x = tf.keras.layers.GRU(gru_units, return_sequences=True)(x)

    # Final output: 1 value per timestep
    output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='softplus'))(x)

    model = tf.keras.Model(inputs=cat_inputs + [cont_input], outputs=output)
    return model


In [35]:
class BaseRegressor(ABC):
    def __init__(
        self, 
        model_fn, 
        model_params, 
        preprocessor_fn = None, 
        cv = 5, 
        cv_group = 'year', 
        alphas = [0.05, 0.95], 
        method = 'indirect',
        n_models = 5,
        cat_features = None
    ):
        self.model_fn = model_fn
        self.model_params = model_params
        self.preprocessor_fn = preprocessor_fn
        self.cv = cv
        self.cv_group = cv_group
        self.alphas = alphas
        self.method = method
        self.n_models = n_models
        self.cat_features = cat_features
        self.lr = self.model_params.pop('lr') if 'lr' in self.model_params.keys() else 0.01

        self.preprocessors = {}
        self.models = {}
        self.history = {}

    @abstractmethod
    def fit(self, X, y):           
        raise NotImplementedError

    @abstractmethod
    def predict(self, X):
        raise NotImplementedError

    @abstractmethod
    def save_model(self, save_path):
        raise NotImplementedError

    @abstractmethod
    def load_model(self, load_path):
        raise NotImplementedError

    @abstractmethod
    def prep_data_for_model(self, X):
        raise NotImplementedError

    def get_data(self, X, y, train_ids, val_ids):
        X_train, y_train = X.iloc[train_ids], y.iloc[y_train]
        X_val, y_val = X.iloc[val_ids], y.iloc[val_ids]
        return X_train, X_val, y_train, y_val

    def cv_split(self, X, group):
        cgkf = ContiguousGroupKFold(self.cv)
        return cgkf.split(X, groups = group)
        

In [36]:
class DeepEnsembleRegressor(BaseRegressor):
    def __init__(self, *args, early_stopping = 3, save_best_weight = True, **kwargs):
        super().__init__(*args, **kwargs)
        self.early_stopping = early_stopping
        self.save_best_weight = save_best_weight

    def fit(self, X, y, batch_size = 32, epochs = 100):
        # split the data into folds
        splits = self.cv_split(X, X[self.cv_group])

        # loop through each fold
        for idx, (train_ids, val_ids) in enumerate(splits):
            # get fold data
            X_train, X_val, y_train, y_val = self.get_data(X, y, train_ids, val_ids)
            
            # preprocess data
            preprocessor = self.preprocessor_fn()
            X_train = preprocessor.fit_transform(X_train)
            X_val = preprocessor.transform(X_val)

            X_train = self.prep_data_for_model(X_train)
            X_val = self.prep_data_for_model(X_val)

            # save preprocessor
            self.preprocessors[f'fold_{idx}'] = preprocessor

            # get the type of model to build, direct or indirect
            if self.method == 'direct':
                self.n_models = 3
                losses = [quantile_loss(self.alphas[0]), smooth_loss(), quantile_loss(self.alphas[-1])]
                for i in range(self.n_models):
                    # create model
                    model = self.model_fn(self.model_params)
                    # create callbacks
                    if self.early_stopping:
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor_loss = 'val_loss', patience = self.early_stopping, restore_best_weights = self.use_best_weight)]
                    else:
                        callbacks = None
                    # compile model
                    model.compile(loss = losses[i], optimizer = tf.keras.optimizers.Adam(learning_rate = self.lr), metrics = ['mae', 'mse', non_negative_log_loss])
                    self.history[f'fold_{idx}_model_{i}'] = model.fit(
                        X_train, y_train, 
                        batch_size = batch_size, 
                        epochs = epochs, 
                        validation_data = [X_val, y_val],
                        callbacks = callbacks
                    )
                    # store model
                    self.models[f'fold_idx_model_{i}'] = model

            elif self.method == 'indirect':
                for i in range(self.n_models):
                    # create model
                    model = self.model_fn(self.model_params)
                    # create callbacks
                    if self.early_stopping:
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor_loss = 'val_loss', patience = self.early_stopping, restore_best_weights = self.use_best_weight)]
                    else:
                        callbacks = None
                    # compile model
                    model.compile(loss = smooth_loss(), optimizer = tf.keras.optimizers.Adam(learning_rate = self.lr), metrics = ['mae', 'mse', non_negative_log_loss])
                    self.history[f'fold_{idx}_model_{i}'] = model.fit(
                        X_train, y_train, 
                        batch_size = batch_size, 
                        epochs = epochs, 
                        validation_data = [X_val, y_val],
                        callbacks = callbacks
                    )
                    # store model
                    self.models[f'fold_idx_model_{i}'] = model

    def predict(self, X):
        raise NotImplementedError

    def save_model(self, save_path):
        raise NotImplementedError

    def load_model(self, load_path):
        raise NotImplementedError
                
    def prep_data_for_model(self, X):
        # for deep learning convert categorical and continuous into a list of inputs
        raise NotImplementedError


In [37]:
model = create_sequence_generator_model(
    cat_input_shapes=[10, 15, 20, 6],
    cont_input_shape=200,
    output_shape=4,
    embedding_dim = 64,
    gru_units = 128, # Generate a sequence of 4 outputs
)
model.summary()


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cat_input_0         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_1         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_2         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_3         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_0         │ (None, 1, 64)     │        640 │ cat_input_0[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 64)     │        960 │ cat_input_1[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 64)     │      1,280 │ cat_input_2[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 64)     │        384 │ cat_input_3[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_21          │ (None, 64)        │          0 │ embedding_0[0][0] │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_22          │ (None, 64)        │          0 │ embedding_1[0][0] │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_23          │ (None, 64)        │          0 │ embedding_2[0][0] │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_24          │ (None, 64)        │          0 │ embedding_3[0][0] │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ continuous_input    │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 456)       │          0 │ reshape_21[0][0], │
│ (Concatenate)       │                   │            │ reshape_22[0][0], │
│                     │                   │            │ reshape_23[0][0], │
│                     │                   │            │ reshape_24[0][0], │
│                     │                   │            │ continuous_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │     58,368 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_13[0][0]    │
│ (BatchNormalizatio… │                   │            │                 

 Total params: 276,929 (1.06 MB)

 Trainable params: 276,673 (1.06 MB)

 Non-trainable params: 256 (1.00 KB)